In [18]:
#import modules
import pandas as pd
import pandas_datareader as web
from pandas_datareader import data as pdr
import yfinance as yf
import numpy as np
import matplotlib.pyplot as plt
import statistics as st
from scipy.stats import mstats
from scipy import stats
import seaborn as sns
from statsmodels.formula.api import ols
import pingouin as pg
import schedule
import time
import requests
import talib

ModuleNotFoundError: No module named 'talib'

### Collect Historical Data

In [6]:
def collect_historical_data(ticker, start_date, end_date):
    """
    Collect historical stock data for a given ticker and time range.

    Parameters:
        ticker (str): Stock ticker symbol (e.g., 'AAPL')
        start_date (str): Start date in 'YYYY-MM-DD' format
        end_date (str): End date in 'YYYY-MM-DD' format

    Returns:
        pd.DataFrame: Historical stock data
    """
    # Fetch the stock data from Yahoo Finance
    yf.pdr_override()
    stock_data = pdr.get_data_yahoo(ticker, start=start_date, end=end_date)
    stock_data = stock_data.reset_index()

    # Return the data
    return stock_data

### Handle Missing Values

In [7]:
def clean_historical_data(stock_data):
    """
    Clean the stock data by handling missing values, outliers, and standardizing formats.

    Parameters:
        stock_data (pd.DataFrame): Historical stock data to clean

    Returns:
        pd.DataFrame: Cleaned stock data
    """
    # Step 1: Handle Missing Values
    # Forward fill or backward fill missing data
    stock_data.fillna(method='ffill', inplace=True)
    stock_data.fillna(method='bfill', inplace=True)

    # Step 2: Handle Outliers (Optional)
    # Define thresholds for outliers (for example, extreme price fluctuations)
    # You can set a threshold based on z-scores or percentage changes between rows.
    z_scores = (stock_data['Close'] - stock_data['Close'].mean()) / stock_data['Close'].std()
    
    # Removing outliers with z-scores greater than 3 or less than -3 (adjust based on your strategy)
    stock_data = stock_data[(np.abs(z_scores) < 3)]

    # Step 3: Standardize Date/Time Format
    # Ensure the index is in the correct datetime format
    stock_data.index = pd.to_datetime(stock_data.index)

    # Step 4: Drop duplicates if any (ensure no duplicated dates)
    stock_data = stock_data.loc[~stock_data.index.duplicated(keep='first')]

    print("Data after cleaning:\n", stock_data.head())
    
    return stock_data

### Compute Technical Indicators

In [8]:
def compute_technical_indicators(stock_data):
    """
    Inspect and analyze the stock data by visualizing price trends and computing technical indicators.

    Parameters:
        stock_data (pd.DataFrame): Historical stock data to inspect and analyze
    """

    # Calculate SMAs
    stock_data['MA50'] = stock_data['Close'].rolling(window=50).mean()
    stock_data['MA200'] = stock_data['Close'].rolling(window=200).mean()
    
    # Step 2: Calculate and Plot Moving Averages (MA)
    stock_data['EMA20'] = talib.EMA(stock_data['Close'], timeperiod=20)
    stock_data['EMA50'] = talib.EMA(stock_data['Close'], timeperiod=50)
    
    # Step 3: Calculate and Plot Bollinger Bands (Volatility)
    stock_data['BB_upper'], stock_data['BB_middle'], stock_data['BB_lower'] = talib.BBANDS(
        stock_data['Close'], timeperiod=20, nbdevup=2, nbdevdn=2)
    
    # Step 4: Calculate and Plot Relative Strength Index (RSI)
    stock_data['RSI'] = talib.RSI(stock_data['Close'], timeperiod=14)
    
    # Add Stochastic Oscillator
    stock_data['slowk'], stock_data['slowd'] = talib.STOCH(
    stock_data['High'], stock_data['Low'], stock_data['Close'], fastk_period=14, slowk_period=3, slowd_period=3)

    # Step 5: Calculate and Plot Moving Average Convergence Divergence (MACD)
    macd, macdsignal, macdhist = talib.MACD(stock_data['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    stock_data['MACD_Hist'] = macdhist
    
    # ATR
    stock_data['ATR'] = talib.ATR(stock_data['High'], stock_data['Low'], stock_data['Close'], timeperiod=14)

    # ADX
    stock_data['ADX'] = talib.ADX(stock_data['High'], stock_data['Low'], stock_data['Close'], timeperiod=14)

    # OBV (On-Balance Volume)
    stock_data['OBV'] = talib.OBV(stock_data['Close'], stock_data['Volume'])

    # Chaikin Money Flow (CMF)
    stock_data['CMF'] = ((stock_data['Close'] - stock_data['Low']) - (stock_data['High'] - stock_data['Close'])) / (stock_data['High'] - stock_data['Low']) * stock_data['Volume']
    stock_data['CMF'] = stock_data['CMF'].rolling(window=20).mean()
    
    # Step 6: Inspect basic statistics
    return stock_data
    #print("Basic Statistics of Stock Data:\n", stock_data.describe())

### Collect Fundamental Data

In [9]:
def get_fundamental_data(ticker):
    """
    Fetch the most recent fundamental data for the ticker (e.g., quarterly updates).
    """
    stock = yf.Ticker(ticker)
    fundamentals = stock.info
    
    fundamental_data = {
        'P/E Ratio': fundamentals.get('trailingPE'),
        'EPS': fundamentals.get('trailingEps'),
        'Dividend Yield': fundamentals.get('dividendYield'),
        'Price-to-Book': fundamentals.get('priceToBook'),
        'Market Cap': fundamentals.get('marketCap'),
        'Revenue': fundamentals.get('totalRevenue'),
        'Net Income': fundamentals.get('netIncomeToCommon')
    }
    return fundamental_data

### Fuse Historical and Fundamental Data

In [10]:
def fuse_price_and_fundamental_data(stock_data, fundamental_data):
    """
    Add fundamental data to the daily price data DataFrame, so every row has the same fundamental values.
    """
    # Convert the fundamental data dictionary into a DataFrame with the same index as the price data
    fundamentals_df = pd.DataFrame([fundamental_data] * len(stock_data), index=stock_data.index)
    
    # Concatenate the price data and fundamental data
    fused_data = pd.concat([stock_data, fundamentals_df], axis=1)
    
    return fused_data

### Collect Macro Data

In [11]:
def get_fred_data(series_id, api_key):
    """
    Fetch macroeconomic data from FRED.
    """
    url = f'https://api.stlouisfed.org/fred/series/observations?series_id={series_id}&api_key={api_key}&file_type=json'
    response = requests.get(url)
    data = response.json()['observations']
    df = pd.DataFrame(data)
    df['date'] = pd.to_datetime(df['date'])
    df.set_index('date', inplace=True)
    df['value'] = pd.to_numeric(df['value'])
    return df

def get_vix_data(start_date, end_date):
    """
    Fetch VIX (Volatility Index) data from Yahoo Finance.
    """
    vix_data = yf.download('^VIX', start=start_date, end=end_date)
    return vix_data

def get_oil_prices_data(start_date, end_date):
    """
    Fetch WTI Crude Oil Prices from Yahoo Finance.
    """
    oil_data = yf.download('CL=F', start=start_date, end=end_date)
    return oil_data

In [12]:
def forward_fill_macro_data(macro_data):
    """
    Forward-fill macroeconomic data for daily alignment.
    """
    return macro_data.reindex(pd.date_range(macro_data.index.min(), macro_data.index.max(), freq='D')).ffill()

In [13]:
def merge_macro_with_stock(fused_data, macro_data, column_name):
    """
    Merge a single macroeconomic dataset with the stock data.
    
    Parameters:
        fused_data (pd.DataFrame): The historical stock and fundamental dataset.
        macro_data (pd.DataFrame): The macroeconomic data to merge.
        column_name (str): Name for the new column in the merged dataset.
        
    Returns:
        pd.DataFrame: Fused dataset with the macroeconomic data added.
    """
    # Ensure macro data has the correct column name for merging
    macro_data = macro_data.rename(columns={'value': column_name})
    
    # Merge macro data with stock data on the index (date)
    merged_data = pd.merge(fused_data, macro_data[[column_name]], how='left', left_index=True, right_index=True)
    
    return merged_data

### Calculate Z-Scores

In [14]:
def calculate_z_score(df, window=20):
    """
    Calculate the Z-Score of the closing price over a given rolling window.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing stock price data.
        window (int): Rolling window size (e.g., 20 days).
    
    Returns:
        pd.Series: Z-Scores of the closing price.
    """
    rolling_mean = df['Close'].rolling(window=window).mean()
    rolling_std = df['Close'].rolling(window=window).std()
    
    z_score = (df['Close'] - rolling_mean) / rolling_std
    
    return z_score

### Calculate Fibonacci Retracement Levels

In [15]:
def calculate_fibonacci_levels(df, window=100):
    """
    Calculate Fibonacci retracement levels based on the high and low points over a rolling window.
    
    Parameters:
        df (pd.DataFrame): DataFrame containing stock price data.
        window (int): Rolling window size (e.g., 100 days).
    
    Returns:
        pd.DataFrame: DataFrame with Fibonacci levels (23.6%, 38.2%, 50%, 61.8%, 100%) for each day.
    """
    high = df['High'].rolling(window=window).max()
    low = df['Low'].rolling(window=window).min()
    
    # Fibonacci retracement levels
    fib_236 = high - 0.236 * (high - low)
    fib_382 = high - 0.382 * (high - low)
    fib_50 = high - 0.5 * (high - low)
    fib_618 = high - 0.618 * (high - low)
    fib_100 = high
    
    fib_df = pd.DataFrame({
        'Fib_23.6%': fib_236,
        'Fib_38.2%': fib_382,
        'Fib_50%': fib_50,
        'Fib_61.8%': fib_618,
        'Fib_100%': fib_100
    }, index=df.index)
    
    return fib_df

### Save Data

In [16]:
def save_stock_data(stock_data, ticker):
    """
    Save the stock data to a CSV file.

    Parameters:
        stock_data (pd.DataFrame): Historical stock data
        ticker (str): Stock ticker symbol to use in the filename
    """
    # Save to CSV
    file_name = f"Data/{ticker}_merged_data.csv"
    stock_data.to_csv(file_name)
    print(f"Data saved to {file_name}")

### Main Function

In [17]:
def main():
    # Parameters for data collection
    ticker = 'AAPL'  # Apple Inc.
    start_date = '2013-01-01'  # Start date (5 years of data)
    end_date = '2023-01-01'    # End date
    api_key = 'your_fred_api_key'

    # Step 2: Collect stock data
    stock_data = collect_historical_data(ticker, start_date, end_date)

    # Step 3: Clean price data
    clean_historical_data(stock_data)

    # Step 4: Compute technical indicators
    compute_technical_indicators(stock_data)

    # Step 5: Collect fundamental data
    fundamental_data = get_fundamental_data(ticker)

    # Step 6: Fuse price and fundamental data
    fused_data = fuse_price_and_fundamental_data(stock_data, fundamental_data)

    # Step 7: Collect interest data
    treasury_yield_df = get_fred_data('DGS10', api_key)

    # Step 8: Collect inflation data
    inflation_rate_df = get_fred_data('CPIAUCSL', api_key)

    # Step 9: Collect unemployment data
    unemployment_rate_df = get_fred_data('UNRATE', api_key)

    # Step 10: Collect GDP data
    gdp_growth_df = get_fred_data('A191RL1Q225SBEA', api_key)

    # Step 11: Collect VIX data
    vix_data = get_vix_data(start_date, end_date)

    # Step 12: Collect oil prices data
    oil_data = get_oil_prices_data(start_date, end_date)

    # Step 13: Forward fill macro data
    treasury_yield_df = forward_fill_macro_data(treasury_yield_df)
    inflation_rate_df = forward_fill_macro_data(inflation_rate_df)
    unemployment_rate_df = forward_fill_macro_data(unemployment_rate_df)
    gdp_growth_df = forward_fill_macro_data(gdp_growth_df)
    vix_data = forward_fill_macro_data(vix_data)
    oil_data = forward_fill_macro_data(oil_data)

    # Step 14: Merge macro with stock data
    fused_data = merge_macro_with_stock(fused_data, treasury_yield_df, 'Interest_Rates')
    fused_data = merge_macro_with_stock(fused_data, inflation_rate_df, 'Inflation_Rate')
    fused_data = merge_macro_with_stock(fused_data, unemployment_rate_df, 'Unemployment_Rate')
    fused_data = merge_macro_with_stock(fused_data, gdp_growth_df, 'GDP_Growth')
    fused_data = merge_macro_with_stock(fused_data, vix_data['Close'], 'VIX')
    fused_data = merge_macro_with_stock(fused_data, oil_data['Close'], 'Oil_Prices') 

    # Step 15: Calculate z-scores
    fused_data['Z-Score'] = calculate_z_score(fused_data, window=20)

    # Step 16: Calculate fibonacci levels
    fibonacci_levels_df = calculate_fibonacci_levels(fused_data, window=100)
    fused_data = pd.concat([fused_data, fibonacci_levels_df], axis=1)

    # Step 5: Save the cleaned data
    save_stock_data(fused_data, ticker)

# Run the main function
if __name__ == "__main__":
    main()

C:\Users\aless\AppData\Local\Programs\Python\Python312\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[*********************100%%**********************]  1 of 1 completed
C:\Users\aless\AppData\Local\Temp\ipykernel_28828\501115791.py:13: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='ffill', inplace=True)
C:\Users\aless\AppData\Local\Temp\ipykernel_28828\501115791.py:14: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_data.fillna(method='bfill', inplace=True)


Data after cleaning:
                                     Date       Open       High        Low  \
1970-01-01 00:00:00.000000000 2013-01-02  19.779285  19.821428  19.343929   
1970-01-01 00:00:00.000000001 2013-01-03  19.567142  19.631071  19.321428   
1970-01-01 00:00:00.000000002 2013-01-04  19.177500  19.236786  18.779642   
1970-01-01 00:00:00.000000003 2013-01-07  18.642857  18.903570  18.400000   
1970-01-01 00:00:00.000000004 2013-01-08  18.900356  18.996071  18.616072   

                                   Close  Adj Close     Volume  
1970-01-01 00:00:00.000000000  19.608213  16.705706  560518000  
1970-01-01 00:00:00.000000001  19.360714  16.494837  352965200  
1970-01-01 00:00:00.000000002  18.821428  16.035385  594333600  
1970-01-01 00:00:00.000000003  18.710714  15.941053  484156400  
1970-01-01 00:00:00.000000004  18.761070  15.983951  458707200  


NameError: name 'talib' is not defined